Программа для размеки документов построчно.

In [1]:
from tkinter import *
from tkinter import filedialog

import cv2
import numpy as np
from PIL import Image, ImageTk
import pytesseract

import os
from width_char_row.class_page import Page

C:\Users\danii\scoop\apps\python\current\lib\site-packages\weasyprint\text\fonts.py:62: UserWarning: FontConfig cannot load default config file. Expect ugly output.
  warn('FontConfig cannot load default config file. Expect ugly output.')
C:\Users\danii\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
PATH = r"C:/Users/danii/Dropbox/Работа/04.Данные/ВКР/"
WIDTH = 500
HEIGHT =700

OFFSET_ROW = 2
BOLD_ROW = 1
REGULAR_ROW = 0

COLOR_OFFSET_ROW = (0, 0, 255) 
COLOR_BOLD_ROW = (255, 0, 0)
COLOR_REGULAR_ROW = (0, 255, 0)

files = []
for end in ('.jpg', '.png', '.jpeg'):
    files += [each for each in os.listdir(PATH) if each.endswith(end)]

In [3]:
def read_img(name_file):
    with open(name_file, "rb") as f:
        chunk = f.read()
    chunk_arr = np.frombuffer(chunk, dtype=np.uint8)
    img = cv2.imdecode(chunk_arr, cv2.IMREAD_COLOR)
    return img

In [5]:
class App:
    def __init__(self):
        self.root = Tk()

        # создаем рабочую область
        self.frame = Frame(self.root)
        self.frame.grid()
        
        self.cv_img = None
        self.image = None
        self.photo = None
        self.box = []
        self.name_box = ""
        self.index_file = 0
        # кнопка открытия файла
        self.btn_open_img = Button(self.frame, text="Открыть картинку", 
                                  command=self.my_event_handler)
        
        self.btn_open_img.grid(row=1, column=1)
        
       
        # кнопка следующая картинка 
        self.btn_next = Button(self.frame, text="<",
                                  command=self.my_back)
        self.btn_next.grid(row=2, column=1)
        self.btn_next = Button(self.frame, text=">",
                                  command=self.my_next)
        self.btn_next.grid(row=2, column=2)
         # кнопка сохранения массива
        self.btn_save_box = Button(self.frame, text="Сохранить данные",
                                  command=self.save_box)
        self.btn_save_box.grid(row=3, column=2)
         
        # Добавим изображение
        self.canvas = Canvas(self.root, height=HEIGHT, width=WIDTH)
        self.canvas.bind("<ButtonPress-1>", self.click)
        self.canvas.grid(row=2, column=1)
        
        self.my_event_handler(file_name=files[self.index_file])
        
        self.root.mainloop()
        
        
    def my_next(self):
        self.save_box()
        self.index_file += 1
        if self.index_file == len(files):
            self.index_file = 0
        self.my_event_handler(files[self.index_file])
        
    def my_back(self):
        self.save_box()
        self.index_file -= 1
        if self.index_file == -1:
            self.index_file = len(files)-1
        self.my_event_handler(files[self.index_file]) 
        
    def my_event_handler(self, file_name=None):
        if file_name is None:
            filename = filedialog.askopenfilename(initialdir=PATH, 
                                          title="Select file", 
                                          filetypes=(("img", "*.jpg *.png *.jpeg"),
                                                  ("all files", "*.*")))  
        else:
            filename = PATH + r'/'+ file_name
        print(filename)
        self.dir = os.path.dirname(filename)
        self.name = os.path.basename(filename)
        self.page = Page(filename)        
        self.draw_img()
        
    def draw_img(self):
        h = self.page.img.shape[0]
        w = self.page.img.shape[1]
        
        img_cope = self.page.img.copy()
        if len(self.page.style) == 0:
            self.page.processing_method(0.56, "ps", line=True)
        for i in range(len(self.page.lines)):
            for j in range(len(self.page.lines[i])):
            
                word = self.page.lines[i][j]
                font = self.page.style[i][j]
                color = (155, 155, 155)
                if font == REGULAR_ROW:
                    color = COLOR_REGULAR_ROW
                elif font == BOLD_ROW:
                    color = COLOR_BOLD_ROW
                elif font == OFFSET_ROW:
                    color = COLOR_OFFSET_ROW
                border = 1
            
                cv2.rectangle(img_cope,  (word.x_top_left, word.y_top_left),
                              (word.x_bottom_right, word.y_bottom_right), color, border)
        coef = h/w
        img = cv2.resize(img_cope, (WIDTH, round(coef*WIDTH)))
        
        self.image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        self.photo = ImageTk.PhotoImage(image = self.image)
        self.c_image = self.canvas.create_image(0, 0, anchor='nw', image=self.photo)
        self.canvas.grid(row=2, column=1)
    
    def click(self, e):
        w = self.page.img.shape[1]
        coef = w/WIDTH
        x1 = e.x*coef
        y1 = e.y*coef
        for i in range(len(self.page.lines)):
            for j in range(len(self.page.lines[i])):
                word = self.page.lines[i][j]
                
                font = self.page.style[i][j]
                color = (155, 155, 155)
                if (x1 > word.x_top_left) and (x1< word.x_bottom_right) and \
                (y1 > word.y_top_left) and (y1 < word.y_bottom_right):
                    if font == REGULAR_ROW:
                        self.page.style[i][j] = BOLD_ROW
                    elif font == BOLD_ROW:
                        self.page.style[i][j] = OFFSET_ROW
                    elif font == OFFSET_ROW:
                        self.page.style[i][j] = REGULAR_ROW
                    border = 1
        self.draw_img()
        
    def save_box(self):
        self.page.save_segm()
        
app= App()

C:/Users/danii/Dropbox/Работа/04.Данные/ВКР//b01.jpeg
C:/Users/danii/Dropbox/Работа/04.Данные/ВКР//b02.jpeg


C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\danii\scoop\apps\python\current\lib\site-packages\width_char_row\my_binar.py:91: RuntimeWarning: invalid value encountered in scalar divide
  mu_2 = (sumVal - mu_k) / (omega_2)


C:/Users/danii/Dropbox/Работа/04.Данные/ВКР//b03.jpeg


C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\danii\scoop\apps\python\current\lib\site-packages\width_char_row\my_binar.py:91: RuntimeWarning: invalid value encountered in scalar divide
  mu_2 = (sumVal - mu_k) / (omega_2)


C:/Users/danii/Dropbox/Работа/04.Данные/ВКР//b04.jpeg


...

C:/Users/danii/Dropbox/Работа/04.Данные/ВКР//n04.jpeg


C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\danii\scoop\apps\python\current\lib\site-packages\width_char_row\my_binar.py:91: RuntimeWarning: invalid value encountered in scalar divide
  mu_2 = (sumVal - mu_k) / (omega_2)


C:/Users/danii/Dropbox/Работа/04.Данные/ВКР//n05.jpeg


C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\danii\scoop\apps\python\current\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


C:/Users/danii/Dropbox/Работа/04.Данные/ВКР//b01.jpeg


In [40]:
os.path.join(PATH, files[0])

'C:\\Users\\danii\\Dropbox\\Работа\\04.Данные\\Разное\\аннотация1.png'